In [1]:
from collections import Counter
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

data = np.load('/home/hll/IDS/2020/data/select/zuizhong/data_train.npy')
label= np.load('/home/hll/IDS/2020/data/select/zuizhong/label6_train.npy')

#  SMOTE all minority classes to (number of data set samples / classes)

In [2]:
X=np.array(data)  
b=np.array(label)
bb=b.reshape(b.shape[0],)    
y10 = np.int32(bb)  

In [3]:
X.shape

(3167779, 12)

In [4]:
sorted(Counter(y10).items())

[(0, 2443895), (1, 200334), (2, 359), (3, 458010), (4, 65144), (5, 37)]

In [5]:
from imblearn.over_sampling import SMOTE
import time
time_start = time.time()

guo = 527963  #Oversampling samples

smo = SMOTE(ratio={1:guo,2:guo,3:guo,4:guo,5:guo},random_state=42)

X_smo, y_smo = smo.fit_sample(X, y10)   
print(sorted(Counter(y_smo).items()))

time_end = time.time()
time = time_end - time_start
print("time:",time)

Using TensorFlow backend.


[(0, 2443895), (1, 527963), (2, 527963), (3, 527963), (4, 527963), (5, 527963)]
time: 2295.455605983734


In [6]:
X_smo.shape[0]

5083710

#  Extract Majority class of data

In [7]:
list0 = []  #data-0
list1 = []  #other classes data
list2 = [] #other classes label


for i in range(X_smo.shape[0]):
    if y_smo[i] == 0:
        list0.append(X_smo[i])
    else:
        list1.append(X_smo[i])
        list2.append(y_smo[i])

In [8]:
data0 = np.array(list0)  
data1 = np.array(list1)
label2 = np.array(list2)
# label22 = label2.reshape(label2.shape[0],)  

print("Normal class data shape：",data0.shape)
print("1-5 class data shape：",data1.shape)  
print("1-5 class data shape：",label2.shape)

Normal class data shape： (2443895, 12)
1-5 class data shape： (2639815, 12)
1-5 class data shape： (2639815,)


# # Cluster majority data into  C (total number of classes)

In [9]:
from sklearn.cluster import KMeans
import time
time_start = time.time()

estimator = KMeans(n_clusters=6)
estimator.fit(data0) 

time_end = time.time()
time = time_end - time_start
print("time:",time)

label_pred_0 = estimator.labels_ 

time: 22.22838044166565


In [10]:
sorted(Counter(label_pred_0).items())

[(0, 687683), (1, 258667), (2, 718137), (3, 110457), (4, 117957), (5, 550994)]

In [12]:
label_pred = label_pred_0

# # Select a certain amount of data from each cluster to form a new majority data

In [13]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

s0=s1=s2=s3=s4=s5=0

for i in range(data0.shape[0]):
    if label_pred[i] == 0:
        c0.append(data0[i])
        s0=s0+1
    elif label_pred[i] == 1:
        c1.append(data0[i])
        s1=s1+1
    elif label_pred[i] == 2:
        c2.append(data0[i])
        s2=s2+1
    elif label_pred[i] == 3:
        c3.append(data0[i])
        s3=s3+1
    elif label_pred[i] == 4:
        c4.append(data0[i])
        s4=s4+1
    elif label_pred[i] == 5:
        c5.append(data0[i])
        s5=s5+1


In [14]:
a=87993

In [15]:
del c0[a:len(c0)]
del c1[a:len(c1)]
del c2[a:len(c2)]
del c3[a:len(c3)]
del c4[a:len(c4)]
del c5[a:len(c5)]

In [16]:
c00 = np.array(c0)
c11 = np.array(c1)
c22 = np.array(c2)
c33 = np.array(c3)
c44 = np.array(c4)
c55 = np.array(c5)


In [17]:
q0 = np.concatenate((c00,c11,c22,c33,c44,c55),axis=0)
q0.shape

(527958, 12)

In [18]:
label_zc = np.zeros((q0.shape[0],), dtype=int)
label_zc.shape

(527958,)

In [19]:
data_end = np.concatenate((q0,data1),axis=0)
label_end = np.concatenate((label_zc,label2),axis=0)

In [21]:
sorted(Counter(label_end).items())

[(0, 527958), (1, 527963), (2, 527963), (3, 527963), (4, 527963), (5, 527963)]

In [22]:
label_end = label_end.reshape(label_end.shape[0],1)

In [23]:
#save dataset
np.save("/home/hll/IDS/2020/data/select/zuizhong/K-means+SMOTE_data_train.npy",data_end)
np.save("/home/hll/IDS/2020/data/select/zuizhong/K-means+SMOTE_label_train.npy",label_end)